In [1]:
using Pkg
Pkg.activate("/Users/ryansenne/Documents/GitHub/SSM")

  Activating project at `\Users\ryansenne\Documents\GitHub\SSM`

In [2]:
include("/Users/ryansenne/Documents/Github/SSM//src/SSM.jl")
using Random
using .SSM
using Distributions

### Gaussian GLM Example

In [10]:
# Fit a Gaussian GLM
# Generate some synthetic data
X = rand(10000, 3);  # 100 samples, 3 features;
# Add a column of ones to represent the intercept term
X_concat = hcat(ones(10000), X)
β = [2, 3, -4, 5]
y = X_concat * β + rand(Normal(0, 1), 10000)

# define a GLM
glm = GaussianRegression(X, y)
fit!(glm, LSELoss(), 1000)

4-element Vector{Float64}:
  2.02647729897031
  3.0079791837279575
 -4.041013313373754
  4.991249502322808

### Poisson GLM Example

In [8]:
# Fit a Poisson GLM
# Generate some synthetic data
X = rand(10000, 3);  # 100 samples, 3 features;
# Add a column of ones to represent the intercept term
X_concat = hcat(ones(10000), X)
β = [2, 3, -4, 5]

# Generate some synthetic data
y = Vector{Float64}(rand.(Poisson.(exp.(X_concat * β))))

# define a GLM
glm = PoissonRegression(X, y)
fit!(glm, PoissonLoss(), 1000)


4-element Vector{Float64}:
  2.000992921834151
  3.0020310084238937
 -4.0003594268560185
  4.996621009202876

### Binomial GLM Example (i.e. logistic regression)

In [7]:
# sigmoid function
function sigmoid(x)
    return 1 / (1 + exp(-x))
end

# Fit a Binomial GLM
# Generate some synthetic data
X = rand(10000, 3); # 1000 samples, 3 features;
# Add a column of ones to represent the intercept term
X_concat = hcat(ones(10000), X)
β = [2, 3, -4, 5]

# Generate some synthetic data
y = Vector{Float64}(rand.(Binomial.(1, sigmoid.(X_concat * β))))

# define a GLM
glm = BinomialRegression(X, y)
fit!(glm, CrossEntropyLoss(), 1000)

4-element Vector{Float64}:
  1.9150124819278913
  3.0577957045910296
 -3.8336980061602524
  4.895904304852471